<a href="https://colab.research.google.com/github/matheuslemesam/Bird_Detection-DL/blob/main/Bird_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bird Detection, projeto de detecção de espécies de pássaros utilizando Redes Convolucionais. UnB/FCTE - 2025.2 - Professor Vinicius Rispoli**

**Para começar o projeto, definimos possíveis arquiteturas mais promissoras: entre elas YOLO, U-Net, EfficientNetV2-L, EfficientNet-B4, ConvNeXt-Tiny. A EfficientNetV2-L foi a que mais se destacou pelo fato de ter uma precisão melhor, treinamento mais robusto e uma melhor tecnologia de detecção.**

### **Importação do Dataset do Google Drive: montar o Google Drive para acessar o dataset de pássaros.**

Começamos iniciando a GPU e vendo se foi iniciada.

In [ ]:
!nvidia-smi

Tue Sep 30 20:41:32 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   64C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from google.colab import drive
import os
from pathlib import Path

# Montar o Google Drive
drive.mount('/content/drive')

# Definir caminhos para Google Colab
base_path = '/content/drive/MyDrive/Dataset_Aves'
dataset_path = os.path.join(base_path, 'original')
output_path = os.path.join(base_path, 'augmentation')

# Verificar se o dataset existe
if os.path.exists(dataset_path):
    print(f"Dataset encontrado em: {dataset_path}")

    # Listar as espécies disponíveis
    species = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]
    print(f"Espécies encontradas ({len(species)}): {species}")

    # Contar imagens por espécie
    total_images = 0
    for specie in species:
        specie_path = os.path.join(dataset_path, specie)
        img_count = len([f for f in os.listdir(specie_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
        print(f"   {specie}: {img_count} imagens")
        total_images += img_count

    print(f"\nTotal de imagens no dataset: {total_images}")
    print(f"Pasta de entrada: {dataset_path}")
    print(f"Pasta de saída: {output_path}")

else:
    print(f"Dataset não encontrado em: {dataset_path}")
    print("Verifique o caminho do dataset")

Mounted at /content/drive
Dataset encontrado em: /content/drive/MyDrive/Dataset_Aves/original
Espécies encontradas (14): ['amazona_amazonica', 'ara_macao', 'amazona_aestiva', 'forpus_xanthopterygius', 'primolius_maracana', 'anodorhynchus_hyacinthinus', 'brotogeris_chiriri', 'diopsittaca_nobilis', 'psittacara_leucophthalmus', 'ara_ararauna', 'touit_melanonotus', 'ara_chloropterus', 'orthopsittaca_manilatus', 'eupsittula_aurea']
   amazona_amazonica: 189 imagens
   ara_macao: 172 imagens
   amazona_aestiva: 240 imagens
   forpus_xanthopterygius: 215 imagens
   primolius_maracana: 195 imagens
   anodorhynchus_hyacinthinus: 250 imagens
   brotogeris_chiriri: 186 imagens
   diopsittaca_nobilis: 217 imagens
   psittacara_leucophthalmus: 222 imagens
   ara_ararauna: 240 imagens
   touit_melanonotus: 150 imagens
   ara_chloropterus: 203 imagens
   orthopsittaca_manilatus: 184 imagens
   eupsittula_aurea: 216 imagens

Total de imagens no dataset: 2879
Pasta de entrada: /content/drive/MyDrive/Da

## **Data Augmentation: aumentar os dados de forma artificial, neste caso com rotações, translações e espelhamento.**

Para o data augmentation, primeiramente importamos as bibliotecas necessárias, Pytorch e Keras.

In [ ]:
!pip install -U torch torchvision torchaudio
!pip install -U keras scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 132.4 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
  Attempting uninstall: keras
    Found existing installation: keras 3.10.0
    Uninstalling keras-3.10.0:
      Successfully uninstalled keras-3.10.0


In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
from PIL import Image
import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

def rotate_image_torch(image, angle):
    """Rotaciona a imagem por um ângulo específico usando PyTorch"""
    return TF.rotate(image, angle, fill=0)

def flip_horizontal_torch(image):
    """Espelha a imagem horizontalmente usando PyTorch"""
    return TF.hflip(image)

def translate_image_torch(image, tx, ty):
    """Translada a imagem usando PyTorch"""
    return TF.affine(image, angle=0, translate=[tx, ty], scale=1, shear=0, fill=0)

def load_image_pil(path):
    """Carrega imagem como PIL Image"""
    return Image.open(path).convert('RGB')

def save_image_pil(image, path):
    """Salva a imagem PIL no caminho especificado"""
    image.save(path, 'JPEG', quality=95)

# Definir transformações base
base_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Redimensionar para tamanho padrão
    # Não aplicamos ToTensor aqui pois queremos manter como PIL Image para salvar
])

print("Funções de augmentation PyTorch carregadas com sucesso!")

Funções de augmentation PyTorch carregadas com sucesso!


In [ ]:
def augment_bird_dataset_torch(input_dir, output_dir):

    # Criar diretório de saída principal
    os.makedirs(output_dir, exist_ok=True)

    # Ângulos de rotação (de 30 em 30 graus)
    angles = [30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330]

    # Translações (pixels de deslocamento)
    translations = [
        (50, 0),    # direita
        (-50, 0),   # esquerda
        (0, 50),    # baixo
        (0, -50),   # cima
        (35, 35),   # diagonal inferior direita
        (-35, 35),  # diagonal inferior esquerda
        (35, -35),  # diagonal superior direita
        (-35, -35), # diagonal superior esquerda
    ]

    total_images = 0
    total_augmented = 0

    # Processar cada espécie (cada pasta dentro de input_dir)
    species_dirs = [d for d in os.listdir(input_dir) if os.path.isdir(os.path.join(input_dir, d))]

    print(f"Espécies encontradas para augmentation: {len(species_dirs)}")
    for species in species_dirs:
        species_path = os.path.join(input_dir, species)
        img_count = len([f for f in os.listdir(species_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
        print(f"   {species}: {img_count} imagens")

    for species in tqdm(species_dirs, desc="Processando espécies"):
        species_input_dir = os.path.join(input_dir, species)
        species_output_dir = os.path.join(output_dir, species)  # Mantém a mesma estrutura de pastas
        os.makedirs(species_output_dir, exist_ok=True)

        # Listar todas as imagens da espécie atual
        image_files = [f for f in os.listdir(species_input_dir)
                      if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

        print(f"\nProcessando {species}: {len(image_files)} imagens")

        for img_file in tqdm(image_files, desc=f"Augmentando {species}", leave=False):
            img_path = os.path.join(species_input_dir, img_file)
            img_name = os.path.splitext(img_file)[0]

            try:
                # Carregar imagem como PIL Image
                image = load_image_pil(img_path)
                # Aplicar resize padrão
                image = base_transform(image)

                total_images += 1
                augmentation_count = 0

                # 1. ROTAÇÕES DA IMAGEM ORIGINAL (11 augmentações)
                for angle in angles:
                    rotated = rotate_image_torch(image, angle)
                    output_filename = os.path.join(species_output_dir, f"{img_name}_rot_{angle}.jpg")
                    save_image_pil(rotated, output_filename)
                    augmentation_count += 1

                # 2. ESPELHAMENTO HORIZONTAL DA IMAGEM ORIGINAL (1 augmentação)
                flipped = flip_horizontal_torch(image)
                output_filename = os.path.join(species_output_dir, f"{img_name}_flip_h.jpg")
                save_image_pil(flipped, output_filename)
                augmentation_count += 1

                # 3. ROTAÇÕES DA IMAGEM ESPELHADA (11 augmentações)
                for angle in angles:
                    rotated_flipped = rotate_image_torch(flipped, angle)
                    output_filename = os.path.join(species_output_dir, f"{img_name}_flip_rot_{angle}.jpg")
                    save_image_pil(rotated_flipped, output_filename)
                    augmentation_count += 1

                # 4. TRANSLAÇÕES DA IMAGEM ORIGINAL (8 augmentações)
                for i, (tx, ty) in enumerate(translations):
                    translated = translate_image_torch(image, tx, ty)
                    output_filename = os.path.join(species_output_dir, f"{img_name}_trans_{i+1}.jpg")
                    save_image_pil(translated, output_filename)
                    augmentation_count += 1

                # 5. TRANSLAÇÕES DA IMAGEM ESPELHADA (8 augmentações)
                for i, (tx, ty) in enumerate(translations):
                    translated_flipped = translate_image_torch(flipped, tx, ty)
                    output_filename = os.path.join(species_output_dir, f"{img_name}_flip_trans_{i+1}.jpg")
                    save_image_pil(translated_flipped, output_filename)
                    augmentation_count += 1

                total_augmented += augmentation_count

            except Exception as e:
                print(f"Erro ao processar {img_path}: {str(e)}")
                continue

    print(f"\nData Augmentation concluído")
    print(f"Imagens originais processadas: {total_images}")
    print(f"Total de augmentações geradas: {total_augmented}")
    print(f"Fator de aumento: {total_augmented/total_images:.1f}x por imagem")
    print(f"Dataset augmentado salvo em: {output_dir}")
    print(f"\nEstrutura criada:")
    print(f"   {output_dir}/")
    for species in [d for d in os.listdir(input_dir) if os.path.isdir(os.path.join(input_dir, d))]:
        print(f"   ├── {species}/")

print("Função de augmentation PyTorch preparada para estrutura Dataset_Aves/")

Função de augmentation PyTorch preparada para estrutura Dataset_Aves/


In [ ]:
import os
import shutil
from pathlib import Path
import time # Importar módulo time para adicionar delays

def fluxo_aumento_dados_robusto(caminho_dataset, caminho_saida):
    """
    Executa aumento de dados de forma robusta, processando por espécie,
    lidando com reconexão do Google Drive e salvando no Drive por espécie.
    """

    print("INICIANDO FLUXO DE AUMENTO DE DADOS ROBUSTO (POR ESPÉCIE)")
    print(f"Origem (dataset original): {caminho_dataset}")
    print(f"Destino final (dataset aumentado no Drive): {caminho_saida}")
    print("-" * 60)

    # Limpar o diretório de saída principal no Drive antes de começar, se existir
    print(f"Preparando destino principal no Google Drive...")
    if os.path.exists(caminho_saida):
        print(f"   Removendo dataset antigo: {caminho_saida}")
        shutil.rmtree(caminho_saida)
        print("   Dataset antigo removido.")
    os.makedirs(caminho_saida, exist_ok=True)
    print(f"   Diretório de destino principal criado: {caminho_saida}")
    print("-" * 60)

    # Obter a lista de espécies no diretório original
    diretorios_especies = [d for d in os.listdir(caminho_dataset) if os.path.isdir(os.path.join(caminho_dataset, d))]
    print(f"Total de espécies a serem processadas: {len(diretorios_especies)}")
    print(f"Espécies: {diretorios_especies}")
    print("-" * 60)

    for i, especie in enumerate(diretorios_especies):
        diretorio_entrada_especie = os.path.join(caminho_dataset, especie)
        diretorio_saida_especie_drive = os.path.join(caminho_saida, especie) # Diretório de saída para a espécie no Drive

        # 1. DEFINIR CAMINHO TEMPORÁRIO LOCAL PARA ESTA ESPÉCIE
        caminho_temporario_especie = f'/content/augmentation_temp/{especie}' # Caminho temporário por espécie

        print(f"\n--- PROCESSANDO ESPÉCIE: {especie} ({i+1}/{len(diretorios_especies)}) ---")
        print(f"   Origem da espécie: {diretorio_entrada_especie}")
        print(f"   Processamento local temporário: {caminho_temporario_especie}")
        print(f"   Destino da espécie no Drive: {diretorio_saida_especie_drive}")
        print("-" * 30)


        try:
            # 2. LIMPAR AMBIENTE LOCAL PARA ESTA ESPÉCIE
            print(f"Limpando ambiente local para {especie}...")
            if os.path.exists(caminho_temporario_especie):
                shutil.rmtree(caminho_temporario_especie)
                print(f"   Diretório temporário removido: {caminho_temporario_especie}")

            os.makedirs(caminho_temporario_especie, exist_ok=True)
            print(f"   Diretório temporário criado: {caminho_temporario_especie}")

            # 3. VERIFICAR ESPAÇO EM DISCO LOCAL (Geral, antes de cada espécie)
            print("\nVerificando espaço em disco local...")
            total, usado, livre = shutil.disk_usage('/content')
            livre_gb = livre / (1024**3)
            print(f"   Espaço livre no /content: {livre_gb:.1f} GB")

            if livre_gb < 5.0:  # Menos que 5GB por segurança
                 print("   AVISO: Pouco espaço livre. O processo pode falhar.")
                 # Adicionar uma pausa ou um prompt para o usuário aqui pode ser útil
                 # input("Pressione Enter para continuar ou Crtl+C para cancelar...") # Exemplo de pausa

            else:
                 print("   Espaço suficiente para processamento local.")


            # 4. EXECUTAR PROCESSAMENTO LOCAL PARA ESTA ESPÉCIE
            print(f"\nINICIANDO AUMENTO DE DADOS LOCAL para {especie}...")
            print(f"   Lendo de: {diretorio_entrada_especie} (Google Drive)")
            print(f"   Escrevendo em: {caminho_temporario_especie} (Local - SSD rápido)")

            # Lógica de augmentation (copiada/adaptada da augment_bird_dataset_torch)
            angulos = [30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330]
            translacoes = [
                (50, 0), (-50, 0), (0, 50), (0, -50),
                (35, 35), (-35, 35), (35, -35), (-35, -35),
            ]
            transformacao_base = transforms.Compose([transforms.Resize((224, 224))]) # Redimensionar

            os.makedirs(caminho_temporario_especie, exist_ok=True) # Garantir que o dir de saída da espécie existe localmente

            arquivos_imagem = [f for f in os.listdir(diretorio_entrada_especie)
                           if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

            total_imagens_especie = 0
            total_aumentadas_especie = 0

            print(f"\nAumentando {especie}: {len(arquivos_imagem)} imagens originais")

            for arquivo_img in tqdm(arquivos_imagem, desc=f"Aumentando {especie}", leave=False):
                caminho_img = os.path.join(diretorio_entrada_especie, arquivo_img)
                nome_img = os.path.splitext(arquivo_img)[0]

                try:
                    imagem = load_image_pil(caminho_img) # Carregar imagem como PIL Image
                    imagem = transformacao_base(imagem)    # Aplicar resize

                    total_imagens_especie += 1
                    contagem_aumento = 0

                    # ROTAÇÕES DA IMAGEM ORIGINAL
                    for angulo in angulos:
                        rotacionada = rotate_image_torch(imagem, angulo)
                        nome_arquivo_saida = os.path.join(caminho_temporario_especie, f"{nome_img}_rot_{angulo}.jpg")
                        save_image_pil(rotacionada, nome_arquivo_saida)
                        contagem_aumento += 1

                    # ESPELHAMENTO HORIZONTAL DA IMAGEM ORIGINAL
                    espelhada = flip_horizontal_torch(imagem)
                    nome_arquivo_saida = os.path.join(caminho_temporario_especie, f"{nome_img}_flip_h.jpg")
                    save_image_pil(espelhada, nome_arquivo_saida)
                    contagem_aumento += 1

                    # ROTAÇÕES DA IMAGEM ESPELHADA
                    for angulo in angulos:
                        rotacionada_espelhada = rotate_image_torch(espelhada, angulo)
                        nome_arquivo_saida = os.path.join(caminho_temporario_especie, f"{nome_img}_flip_rot_{angulo}.jpg")
                        save_image_pil(rotacionada_espelhada, nome_arquivo_saida)
                        contagem_aumento += 1

                    # TRANSLAÇÕES DA IMAGEM ORIGINAL
                    for k, (tx, ty) in enumerate(translacoes):
                        transladada = translate_image_torch(imagem, tx, ty)
                        nome_arquivo_saida = os.path.join(caminho_temporario_especie, f"{nome_img}_trans_{k+1}.jpg")
                        save_image_pil(transladada, nome_arquivo_saida)
                        contagem_aumento += 1

                    # TRANSLAÇÕES DA IMAGEM ESPELHADA
                    for k, (tx, ty) in enumerate(translacoes):
                        transladada_espelhada = translate_image_torch(espelhada, tx, ty)
                        nome_arquivo_saida = os.path.join(caminho_temporario_especie, f"{nome_img}_flip_trans_{k+1}.jpg")
                        save_image_pil(transladada_espelhada, nome_arquivo_saida)
                        contagem_aumento += 1

                    total_aumentadas_especie += contagem_aumento

                except Exception as e:
                    print(f"Erro ao processar {caminho_img}: {str(e)}")
                    continue

            print(f"   Aumento de dados local para {especie} concluído.")
            print(f"   Imagens originais: {total_imagens_especie}, Aumentadas: {total_aumentadas_especie}")
            print(f"   Fator de aumento: {total_aumentadas_especie/total_imagens_especie if total_imagens_especie > 0 else 0:.1f}x")
            print("-" * 30)


            # 5. VERIFICAR CONEXÃO E RECONECTAR SE NECESSÁRIO
            print(f"\nVerificando conexão com Google Drive para {especie}...")
            drive_montado = os.path.exists('/content/drive')
            while not drive_montado:
                print("   Google Drive não está montado ou foi desconectado.")
                print("   Tentando reconectar...")
                try:
                    drive.mount('/content/drive', force_remount=True)
                    drive_montado = os.path.exists('/content/drive')
                    if drive_montado:
                        print("   Google Drive reconectado com sucesso.")
                        # Pode ser útil adicionar uma pequena pausa após reconectar
                        time.sleep(5)
                    else:
                         print("   Falha na reconexão. Verifique as permissões ou tente manualmente.")
                         # Adicionar um prompt para o usuário ou sair se a reconexão falhar repetidamente
                         input("Pressione Enter após reconectar o Drive manualmente ou Crtl+C para cancelar...")
                         drive_montado = os.path.exists('/content/drive') # Verificar novamente após o prompt
                except Exception as e:
                    print(f"   Erro durante a tentativa de reconexão: {str(e)}")
                    input("   Pressione Enter para tentar novamente ou Crtl+C para cancelar...")
                    drive_montado = os.path.exists('/content/drive') # Verificar novamente após o prompt

            print("   Google Drive está conectado.")
            print("-" * 30)


            # 6. COPIAR PARA O DRIVE (APENAS ESTA ESPÉCIE)
            print(f"\nCOPIANDO DADOS AUMENTADOS DE {especie} PARA GOOGLE DRIVE...")
            print(f"   De: {caminho_temporario_especie}")
            print(f"   Para: {diretorio_saida_especie_drive}")
            print("   Esta operação pode demorar um pouco...")

            # Limpar o diretório de destino da espécie no Drive antes de copiar
            if os.path.exists(diretorio_saida_especie_drive):
                print(f"   Removendo diretório antigo da espécie no Drive: {diretorio_saida_especie_drive}")
                shutil.rmtree(diretorio_saida_especie_drive)
                print("   Diretório antigo da espécie removido.")

            os.makedirs(os.path.dirname(diretorio_saida_especie_drive), exist_ok=True) # Garantir diretório pai existe
            shutil.copytree(caminho_temporario_especie, diretorio_saida_especie_drive) # Copiar a pasta da espécie

            print(f"   CÓPIA DE {especie} PARA GOOGLE DRIVE CONCLUÍDA!")
            print("-" * 30)


            # 7. VERIFICAR RESULTADO NO DRIVE (APENAS ESTA ESPÉCIE)
            print(f"\nVerificando resultado final de {especie} no Drive...")
            if os.path.exists(diretorio_saida_especie_drive):
                contagem_img_drive = len([f for f in os.listdir(diretorio_saida_especie_drive)
                                       if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
                print(f"   {especie} no Drive: {contagem_img_drive} imagens")

                if contagem_img_drive == total_aumentadas_especie:
                    print(f"   INTEGRIDADE VERIFICADA para {especie}: Todos os arquivos foram copiados corretamente!")
                else:
                    print(f"   AVISO: Possível perda de dados durante a cópia de {especie}!")
                    print(f"      Local: {total_aumentadas_especie} imagens, Drive: {contagem_img_drive} imagens")
            else:
                print(f"   ERRO: O diretório de destino no Drive para {especie} não foi encontrado após a cópia!")

            print("-" * 30)

            # 8. LIMPAR ARQUIVOS TEMPORÁRIOS LOCAIS PARA ESTA ESPÉCIE
            print(f"\nLimpando arquivos temporários locais para {especie}...")
            if os.path.exists(caminho_temporario_especie):
                shutil.rmtree(caminho_temporario_especie)
                print(f"   Arquivos temporários de {especie} removidos.")
            print("-" * 30)


        except Exception as e:
            print(f"\nERRO DURANTE O PROCESSAMENTO DE {especie}:")
            print(f"   {str(e)}")
            print(f"\nPulando para a próxima espécie após tentativa de limpeza de emergência...")

            # Limpeza de emergência para a espécie atual
            try:
                if os.path.exists(caminho_temporario_especie):
                    shutil.rmtree(caminho_temporario_especie)
                    print(f"   Arquivos temporários de {especie} removidos.")
            except:
                print(f"   Não foi possível remover arquivos temporários de {especie}")

            # Continue para a próxima espécie
            continue

    # 9. RESUMO FINAL (Após processar todas as espécies)
    print("\n" + "="*60)
    print("FLUXO GERAL CONCLUÍDO!")
    print(f"Dataset aumentado disponível em: {caminho_saida}")

    # Contar total final no Drive
    total_imagens_final_drive = 0
    especies_final_drive = [d for d in os.listdir(caminho_saida)
                           if os.path.isdir(os.path.join(caminho_saida, d))]

    print(f"\nResumo final no Drive ({caminho_saida}):")
    for especie in especies_final_drive:
        caminho_especie = os.path.join(caminho_saida, especie)
        contagem_img = len([f for f in os.listdir(caminho_especie)
                         if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
        total_imagens_final_drive += contagem_img
        print(f"   ├── {especie}: {contagem_img} imagens")

    print(f"\nTotal final de imagens no Drive: {total_imagens_final_drive}")
    print(f"Total de espécies no Drive: {len(especies_final_drive)}")

    # Verificar se o número de espécies no Drive corresponde ao original
    if len(especies_final_drive) == len(diretorios_especies):
         print("   VERIFICADO: Número de espécies no Drive corresponde ao original.")
    else:
         print("   AVISO: O número de espécies no Drive não corresponde ao original!")


    print("="*60)


# Função auxiliar para verificar espaço em disco (mantida)
def verificar_espaco_disco():
    """Verifica o espaço disponível no disco"""
    total, usado, livre = shutil.disk_usage('/')

    print(f"Espaço em disco:")
    print(f"Total: {total // (2**30):.1f} GB")
    print(f"Usado: {usado // (2**30):.1f} GB")
    print(f"Livre: {livre // (2**30):.1f} GB")

    if livre < 5 * (2**30):  # Aumentado para 5GB para safety
        print("AVISO: Pouco espaço livre. O augmentation pode falhar.")
    else:
        print("Espaço suficiente para data augmentation local.")

# --- Execução do Workflow ---

# Verificar espaço primeiro
verificar_espaco_disco()

# Executar o workflow robusto, se as variáveis estiverem definidas
if 'dataset_path' in globals() and 'output_path' in globals():
    # Verificar se o diretório de entrada original realmente existe
    if os.path.exists(dataset_path):
        print(f"\nUSANDO PROCESSAMENTO LOCAL (POR ESPÉCIE) + CÓPIA FINAL PARA DRIVE")
        print(f"   Isso resolve problemas de sincronização com Google Drive")
        print(f"   Velocidade ~10x mais rápida")
        print(f"   Maior resiliência a quedas de conexão do Drive")

        fluxo_aumento_dados_robusto(dataset_path, output_path)
    else:
        print("ERRO: dataset_path não existe!")
        print(f"Caminho verificado: {dataset_path}")
else:
    print("ERRO: Variáveis dataset_path e/ou output_path não estão definidas!")
    print("Execute primeiro as células de configuração do Google Drive.")

Espaço em disco:
Total: 112.0 GB
Usado: 39.0 GB
Livre: 73.0 GB
Espaço suficiente para data augmentation local.

USANDO PROCESSAMENTO LOCAL (POR ESPÉCIE) + CÓPIA FINAL PARA DRIVE
   Isso resolve problemas de sincronização com Google Drive
   Velocidade ~10x mais rápida
   Maior resiliência a quedas de conexão do Drive
INICIANDO FLUXO DE AUMENTO DE DADOS ROBUSTO (POR ESPÉCIE)
Origem (dataset original): /content/drive/MyDrive/Dataset_Aves/original
Destino final (dataset aumentado no Drive): /content/drive/MyDrive/Dataset_Aves/augmentation
------------------------------------------------------------
Preparando destino principal no Google Drive...
   Removendo dataset antigo: /content/drive/MyDrive/Dataset_Aves/augmentation
   Dataset antigo removido.
   Diretório de destino principal criado: /content/drive/MyDrive/Dataset_Aves/augmentation
------------------------------------------------------------
Total de espécies a serem processadas: 14
Espécies: ['amazona_amazonica', 'ara_macao', 'ama

   Aumento de dados local para amazona_amazonica concluído.
   Imagens originais: 189, Aumentadas: 7371
   Fator de aumento: 39.0x
------------------------------

Verificando conexão com Google Drive para amazona_amazonica...
   Google Drive está conectado.
------------------------------

COPIANDO DADOS AUMENTADOS DE amazona_amazonica PARA GOOGLE DRIVE...
   De: /content/augmentation_temp/amazona_amazonica
   Para: /content/drive/MyDrive/Dataset_Aves/augmentation/amazona_amazonica
   Esta operação pode demorar um pouco...
   CÓPIA DE amazona_amazonica PARA GOOGLE DRIVE CONCLUÍDA!
------------------------------

Verificando resultado final de amazona_amazonica no Drive...
   amazona_amazonica no Drive: 7371 imagens
   INTEGRIDADE VERIFICADA para amazona_amazonica: Todos os arquivos foram copiados corretamente!
------------------------------

Limpando arquivos temporários locais para amazona_amazonica...
   Arquivos temporários de amazona_amazonica removidos.
----------------------------

   Aumento de dados local para ara_macao concluído.
   Imagens originais: 172, Aumentadas: 6708
   Fator de aumento: 39.0x
------------------------------

Verificando conexão com Google Drive para ara_macao...
   Google Drive está conectado.
------------------------------

COPIANDO DADOS AUMENTADOS DE ara_macao PARA GOOGLE DRIVE...
   De: /content/augmentation_temp/ara_macao
   Para: /content/drive/MyDrive/Dataset_Aves/augmentation/ara_macao
   Esta operação pode demorar um pouco...
   CÓPIA DE ara_macao PARA GOOGLE DRIVE CONCLUÍDA!
------------------------------

Verificando resultado final de ara_macao no Drive...
   ara_macao no Drive: 6708 imagens
   INTEGRIDADE VERIFICADA para ara_macao: Todos os arquivos foram copiados corretamente!
------------------------------

Limpando arquivos temporários locais para ara_macao...
   Arquivos temporários de ara_macao removidos.
------------------------------

--- PROCESSANDO ESPÉCIE: amazona_aestiva (3/14) ---
   Origem da espécie: /content/d

   Aumento de dados local para amazona_aestiva concluído.
   Imagens originais: 240, Aumentadas: 9360
   Fator de aumento: 39.0x
------------------------------

Verificando conexão com Google Drive para amazona_aestiva...
   Google Drive está conectado.
------------------------------

COPIANDO DADOS AUMENTADOS DE amazona_aestiva PARA GOOGLE DRIVE...
   De: /content/augmentation_temp/amazona_aestiva
   Para: /content/drive/MyDrive/Dataset_Aves/augmentation/amazona_aestiva
   Esta operação pode demorar um pouco...
   CÓPIA DE amazona_aestiva PARA GOOGLE DRIVE CONCLUÍDA!
------------------------------

Verificando resultado final de amazona_aestiva no Drive...
   amazona_aestiva no Drive: 9360 imagens
   INTEGRIDADE VERIFICADA para amazona_aestiva: Todos os arquivos foram copiados corretamente!
------------------------------

Limpando arquivos temporários locais para amazona_aestiva...
   Arquivos temporários de amazona_aestiva removidos.
------------------------------

--- PROCESSANDO ES

   Aumento de dados local para forpus_xanthopterygius concluído.
   Imagens originais: 215, Aumentadas: 8385
   Fator de aumento: 39.0x
------------------------------

Verificando conexão com Google Drive para forpus_xanthopterygius...
   Google Drive está conectado.
------------------------------

COPIANDO DADOS AUMENTADOS DE forpus_xanthopterygius PARA GOOGLE DRIVE...
   De: /content/augmentation_temp/forpus_xanthopterygius
   Para: /content/drive/MyDrive/Dataset_Aves/augmentation/forpus_xanthopterygius
   Esta operação pode demorar um pouco...
   CÓPIA DE forpus_xanthopterygius PARA GOOGLE DRIVE CONCLUÍDA!
------------------------------

Verificando resultado final de forpus_xanthopterygius no Drive...
   forpus_xanthopterygius no Drive: 8385 imagens
   INTEGRIDADE VERIFICADA para forpus_xanthopterygius: Todos os arquivos foram copiados corretamente!
------------------------------

Limpando arquivos temporários locais para forpus_xanthopterygius...
   Arquivos temporários de forpus_

...

   Aumento de dados local para ara_ararauna concluído.
   Imagens originais: 240, Aumentadas: 9360
   Fator de aumento: 39.0x
------------------------------

Verificando conexão com Google Drive para ara_ararauna...
   Google Drive está conectado.
------------------------------

COPIANDO DADOS AUMENTADOS DE ara_ararauna PARA GOOGLE DRIVE...
   De: /content/augmentation_temp/ara_ararauna
   Para: /content/drive/MyDrive/Dataset_Aves/augmentation/ara_ararauna
   Esta operação pode demorar um pouco...
   CÓPIA DE ara_ararauna PARA GOOGLE DRIVE CONCLUÍDA!
------------------------------

Verificando resultado final de ara_ararauna no Drive...
   ara_ararauna no Drive: 9360 imagens
   INTEGRIDADE VERIFICADA para ara_ararauna: Todos os arquivos foram copiados corretamente!
------------------------------

Limpando arquivos temporários locais para ara_ararauna...
   Arquivos temporários de ara_ararauna removidos.
------------------------------

--- PROCESSANDO ESPÉCIE: touit_melanonotus (11/14) 

   Aumento de dados local para touit_melanonotus concluído.
   Imagens originais: 150, Aumentadas: 5850
   Fator de aumento: 39.0x
------------------------------

Verificando conexão com Google Drive para touit_melanonotus...
   Google Drive está conectado.
------------------------------

COPIANDO DADOS AUMENTADOS DE touit_melanonotus PARA GOOGLE DRIVE...
   De: /content/augmentation_temp/touit_melanonotus
   Para: /content/drive/MyDrive/Dataset_Aves/augmentation/touit_melanonotus
   Esta operação pode demorar um pouco...
   CÓPIA DE touit_melanonotus PARA GOOGLE DRIVE CONCLUÍDA!
------------------------------

Verificando resultado final de touit_melanonotus no Drive...
   touit_melanonotus no Drive: 5850 imagens
   INTEGRIDADE VERIFICADA para touit_melanonotus: Todos os arquivos foram copiados corretamente!
------------------------------

Limpando arquivos temporários locais para touit_melanonotus...
   Arquivos temporários de touit_melanonotus removidos.
----------------------------

   Aumento de dados local para ara_chloropterus concluído.
   Imagens originais: 203, Aumentadas: 7917
   Fator de aumento: 39.0x
------------------------------

Verificando conexão com Google Drive para ara_chloropterus...
   Google Drive está conectado.
------------------------------

COPIANDO DADOS AUMENTADOS DE ara_chloropterus PARA GOOGLE DRIVE...
   De: /content/augmentation_temp/ara_chloropterus
   Para: /content/drive/MyDrive/Dataset_Aves/augmentation/ara_chloropterus
   Esta operação pode demorar um pouco...
   CÓPIA DE ara_chloropterus PARA GOOGLE DRIVE CONCLUÍDA!
------------------------------

Verificando resultado final de ara_chloropterus no Drive...
   ara_chloropterus no Drive: 7917 imagens
   INTEGRIDADE VERIFICADA para ara_chloropterus: Todos os arquivos foram copiados corretamente!
------------------------------

Limpando arquivos temporários locais para ara_chloropterus...
   Arquivos temporários de ara_chloropterus removidos.
------------------------------

--- PRO

   Aumento de dados local para orthopsittaca_manilatus concluído.
   Imagens originais: 184, Aumentadas: 7176
   Fator de aumento: 39.0x
------------------------------

Verificando conexão com Google Drive para orthopsittaca_manilatus...
   Google Drive está conectado.
------------------------------

COPIANDO DADOS AUMENTADOS DE orthopsittaca_manilatus PARA GOOGLE DRIVE...
   De: /content/augmentation_temp/orthopsittaca_manilatus
   Para: /content/drive/MyDrive/Dataset_Aves/augmentation/orthopsittaca_manilatus
   Esta operação pode demorar um pouco...
   CÓPIA DE orthopsittaca_manilatus PARA GOOGLE DRIVE CONCLUÍDA!
------------------------------

Verificando resultado final de orthopsittaca_manilatus no Drive...
   orthopsittaca_manilatus no Drive: 7176 imagens
   INTEGRIDADE VERIFICADA para orthopsittaca_manilatus: Todos os arquivos foram copiados corretamente!
------------------------------

Limpando arquivos temporários locais para orthopsittaca_manilatus...
   Arquivos temporários 

   Aumento de dados local para eupsittula_aurea concluído.
   Imagens originais: 216, Aumentadas: 8424
   Fator de aumento: 39.0x
------------------------------

Verificando conexão com Google Drive para eupsittula_aurea...
   Google Drive está conectado.
------------------------------

COPIANDO DADOS AUMENTADOS DE eupsittula_aurea PARA GOOGLE DRIVE...
   De: /content/augmentation_temp/eupsittula_aurea
   Para: /content/drive/MyDrive/Dataset_Aves/augmentation/eupsittula_aurea
   Esta operação pode demorar um pouco...
   CÓPIA DE eupsittula_aurea PARA GOOGLE DRIVE CONCLUÍDA!
------------------------------

Verificando resultado final de eupsittula_aurea no Drive...
   eupsittula_aurea no Drive: 8424 imagens
   INTEGRIDADE VERIFICADA para eupsittula_aurea: Todos os arquivos foram copiados corretamente!
------------------------------

Limpando arquivos temporários locais para eupsittula_aurea...
   Arquivos temporários de eupsittula_aurea removidos.
------------------------------

FLUXO G

## **Treinamento do modelo**

Começamos especificando qual framework de Deep Learning o Keras usará, neste caso o PyTorch.

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "torch"

Após isto importamos o keras e pytorch para a parte das Redes Neurais e numpy, matplotlib, panda e scikit-learn para visualizações de dados.

In [ ]:
import keras
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

Agora, fazemos um debug das importações e tomada de decisão: utilizar GPU caso tenha, caso contrário, utilizar a CPU.

In [ ]:
print(f"Versão do Keras: {keras.__version__}") # Printa a versão do Keras

print(f"Keras está usando o backend: {keras.backend.backend()}") # Printa o framework utilizado pelo Keras
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Verifica se tem GPU para utilizar

print(f"Usando o dispositivo: {device}") # Mostra a GPU/CPU que será utilizada

Versão do Keras: 3.11.3
Keras está usando o backend: torch
Usando o dispositivo: cuda


Agora, importamos a arquitetura do modelo EfficientNetV2 com a rede neural construida camada por camada e o modificamos para utilizarmos apenas a effnetv2_l

In [10]:
"""
Creates a EfficientNetV2 Model as defined in:
Mingxing Tan, Quoc V. Le. (2021).
EfficientNetV2: Smaller Models and Faster Training
arXiv preprint arXiv:2104.00298.
import from https://github.com/d-li14/mobilenetv2.pytorch
"""

import torch
import torch.nn as nn
import math

__all__ = ['effnetv2_l']


def _make_divisible(v, divisor, min_value=None):
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    :param v:
    :param divisor:
    :param min_value:
    :return:
    """
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


# SiLU (Swish) activation function
if hasattr(nn, 'SiLU'):
    SiLU = nn.SiLU
else:
    # For compatibility with old PyTorch versions
    class SiLU(nn.Module):
        def forward(self, x):
            return x * torch.sigmoid(x)


class SELayer(nn.Module):
    def __init__(self, inp, oup, reduction=4):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
                nn.Linear(oup, _make_divisible(inp // reduction, 8)),
                SiLU(),
                nn.Linear(_make_divisible(inp // reduction, 8), oup),
                nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y


def conv_3x3_bn(inp, oup, stride):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
        nn.BatchNorm2d(oup),
        SiLU()
    )


def conv_1x1_bn(inp, oup):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
        nn.BatchNorm2d(oup),
        SiLU()
    )


class MBConv(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio, use_se):
        super(MBConv, self).__init__()
        assert stride in [1, 2]

        hidden_dim = round(inp * expand_ratio)
        self.identity = stride == 1 and inp == oup
        if use_se:
            self.conv = nn.Sequential(
                # pw
                nn.Conv2d(inp, hidden_dim, 1, 1, 0, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                SELayer(inp, hidden_dim),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )
        else:
            self.conv = nn.Sequential(
                # fused
                nn.Conv2d(inp, hidden_dim, 3, stride, 1, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )


    def forward(self, x):
        if self.identity:
            return x + self.conv(x)
        else:
            return self.conv(x)


class EffNetV2(nn.Module):
    def __init__(self, cfgs, num_classes=1000, width_mult=1.):
        super(EffNetV2, self).__init__()
        self.cfgs = cfgs

        # building first layer
        input_channel = _make_divisible(24 * width_mult, 8)
        layers = [conv_3x3_bn(3, input_channel, 2)]
        # building inverted residual blocks
        block = MBConv
        for t, c, n, s, use_se in self.cfgs:
            output_channel = _make_divisible(c * width_mult, 8)
            for i in range(n):
                layers.append(block(input_channel, output_channel, s if i == 0 else 1, t, use_se))
                input_channel = output_channel
        self.features = nn.Sequential(*layers)
        # building last several layers
        output_channel = _make_divisible(1792 * width_mult, 8) if width_mult > 1.0 else 1792
        self.conv = conv_1x1_bn(input_channel, output_channel)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(output_channel, num_classes)

        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = self.conv(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.001)
                m.bias.data.zero_()


def effnetv2_l(**kwargs):
    """
    Constructs a EfficientNetV2-L model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  32,  4, 1, 0],
        [4,  64,  7, 2, 0],
        [4,  96,  7, 2, 0],
        [4, 192, 10, 2, 1],
        [6, 224, 19, 1, 1],
        [6, 384, 25, 2, 1],
        [6, 640,  7, 1, 1],
    ]
    return EffNetV2(cfgs, **kwargs)